In [1]:
model_checkpoint = 'google/mt5-small'

In [2]:
!pip install transformers datasets seacrowd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.7/218.7 kB 16.5 MB/s eta 0:00:00
   ━━

In [3]:
from datasets import load_dataset
dset = load_dataset("SEACrowd/liputan6", trust_remote_code=True)

# Use subsets of the dataset
train_data = dset["train"].select(range(10000))
val_data = dset["validation"].select(range(2000))
test_data = dset["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.23k [00:00<?, ?B/s]

liputan6.py:   0%|          | 0.00/7.52k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments, DataCollatorWithPadding

tokenizer_ft_mt5 = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    # Tokenisasi teks input
    model_inputs = tokenizer_ft_mt5(
        examples['document'],
        max_length=512,
        truncation=True,
        padding='max_length'  # Add padding to max_length
    )
    # Tokenisasi target
    with tokenizer_ft_mt5.as_target_tokenizer():
        labels = tokenizer_ft_mt5(
            examples['summary'],
            max_length=128,
            truncation=True,
            padding='max_length'  # Add padding to max_length
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_dataset_train = train_data.map(tokenize_function)
tokenized_dataset_val = train_data.map(tokenize_function)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [5]:
print(tokenized_dataset_train)
print(tokenized_dataset_val)

Dataset({
    features: ['document', 'id', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})
Dataset({
    features: ['document', 'id', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})


In [6]:
tokenized_dataset_train = tokenized_dataset_train.remove_columns(["document", "id", "summary"])
tokenized_dataset_val = tokenized_dataset_val.remove_columns(["document", "id", "summary"])

In [7]:
print(tokenized_dataset_train)
print(tokenized_dataset_val)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})


In [8]:
# Parameter
learning_rate = 5e-5  # Laju pembelajaran
train_batch_size = 6    # Ukuran batch untuk pelatihan
epochs = 5             # Jumlah epoch
weight_decay = 0.001      # Pengurangan bobot

# Direktori untuk menyimpan hasil
output_directory = f'./results_mt5_{learning_rate}_{train_batch_size}_{epochs}'

In [9]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

model_ft_mt5 = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# Use DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_ft_mt5)

training_args = TrainingArguments(
    output_dir = output_directory,          # direktori untuk menyimpan model
    evaluation_strategy="epoch",     # evaluasi setiap epoch
    learning_rate=learning_rate,
    per_device_train_batch_size=train_batch_size,   # ukuran batch untuk pelatihan
    num_train_epochs=epochs,              # jumlah epoch
    weight_decay=weight_decay,               # pengurangan bobot
)


trainer = Trainer(
    model=model_ft_mt5,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    data_collator=data_collator,  # Use the data collator
)

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,1.609500,0.833604
2,1.008100,0.741713
3,0.903300,0.708375
4,0.877900,0.685901
5,0.852400,0.681021


TrainOutput(global_step=8335, training_loss=2.038834879508473, metrics={'train_runtime': 7646.9676, 'train_samples_per_second': 6.539, 'train_steps_per_second': 1.09, 'total_flos': 2.6437484544e+16, 'train_loss': 2.038834879508473, 'epoch': 5.0})

In [24]:
new_article = dset["test"][1]["document"]
new_summary = dset["test"][1]["summary"]
print(new_article)
print(new_summary)

Liputan6 . com , Bandar Lampung : Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas . Data tersebut diungkapkan Kepala Dinas Kesehatan Kota Bandar Lampung M . Sudarman , baru-baru ini . Menurut Sudarman , Dinas Kesehatan Bandar Lampung mencatat sekitar 51 anak terserang busung lapar yang tersebar di beberapa kecamatan , selama periode 1999 sampai 2001 . Kebanyakan anak penderita busung tersebut berasal dari keluarga yang hidup di bawah garis kemiskinan . Selain kekurangan gizi , komplikasi radang paru-paru juga menjadi satu faktor penyebab kematian anak penderita busung lapar tersebut . Data Dinas Kesehatan menunjukkan pada 1999 , ditemukan 41 anak terserang penyakit busung lapar . Sebagian besar penderita berdomisili di kampung miskin Umbul Kunci . Jumlah penderita busung lapar menurun pada 2000 , yakni hanya sembilan anak . Sedangkan September 2001 , seorang anak meninggal karen

In [20]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_result_mt5 = AutoTokenizer.from_pretrained(model_checkpoint)
model_result_mt5 = AutoModelForSeq2SeqLM.from_pretrained('/content/results_mt5_5e-05_6_5/checkpoint-8335')

# Teks input
input_text = new_article
input_teks = input_text
input_ids = tokenizer_result_mt5.encode(input_teks, return_tensors='pt')

# Menghasilkan ringkasan
output_ids = model_result_mt5.generate(input_ids, max_length=70)
output_text = tokenizer_result_mt5.decode(output_ids[0], skip_special_tokens=True)

# print("Referensi: ", referensi_ringkasan)
print("Ringkasan Hasil Fine Tune:", output_text)

Ringkasan Hasil Fine Tune: Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas .


In [21]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# Calculate BLEU score for this sentence
bleu_score = sentence_bleu([new_summary], output_text)

# Print the results
print(f"Input: {new_article}")
print(f"Predicted: {output_text}")
print(f"Reference: {new_summary}")
print(f"BLEU Score: {bleu_score:.4f}")
print("-" * 50)

Input: Liputan6 . com , Bandar Lampung : Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas . Data tersebut diungkapkan Kepala Dinas Kesehatan Kota Bandar Lampung M . Sudarman , baru-baru ini . Menurut Sudarman , Dinas Kesehatan Bandar Lampung mencatat sekitar 51 anak terserang busung lapar yang tersebar di beberapa kecamatan , selama periode 1999 sampai 2001 . Kebanyakan anak penderita busung tersebut berasal dari keluarga yang hidup di bawah garis kemiskinan . Selain kekurangan gizi , komplikasi radang paru-paru juga menjadi satu faktor penyebab kematian anak penderita busung lapar tersebut . Data Dinas Kesehatan menunjukkan pada 1999 , ditemukan 41 anak terserang penyakit busung lapar . Sebagian besar penderita berdomisili di kampung miskin Umbul Kunci . Jumlah penderita busung lapar menurun pada 2000 , yakni hanya sembilan anak . Sedangkan September 2001 , seorang anak meningga

In [22]:
from nltk.translate.bleu_score import sentence_bleu
# List untuk menyimpan skor BLEU
bleu_scores = []

# Pastikan `dset["test"]` memiliki 10 data
for i in range(10):
    # Teks input dari dataset
    new_article = dset["test"][i]["document"]  # Pastikan `document` adalah kunci yang benar


    # Tokenisasi
    input_teks = new_article
    input_ids = tokenizer_result_mt5.encode(input_teks, return_tensors='pt')

    # Menghasilkan ringkasan
    output_ids = model_result_mt5.generate(input_ids, max_length=70)
    output_text = tokenizer_result_mt5.decode(output_ids[0], skip_special_tokens=True)

    # Hitung BLEU score
    bleu_score = sentence_bleu([dset["test"][i]["summary"]], output_text)  # `hypothesis` diubah menjadi list of words
    bleu_scores.append(bleu_score)

    # Cetak hasil setiap iterasi
    print(f"Input: {new_article}")
    print(f"Predicted: {output_text}")
    print(f"Reference: {dset['test'][i]['summary']}")
    print(f"BLEU Score: {bleu_score:.4f}")
    print("-" * 50)

# Hitung rata-rata BLEU score
average_bleu_score = sum(bleu_scores) / len(bleu_scores)

# Cetak rata-rata BLEU score
print(f"Average BLEU Score for 10 samples: {average_bleu_score:.4f}")


Input: Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum patroli datang , mereka telah memindahkan puluh